In [69]:
print('Hola')

Hola


In [70]:
import numpy as np
import pandas as pd
from pathlib import Path

In [71]:
put=Path().resolve()
put2=Path().resolve()/'диаф_48'

#вводим переменные, служащие хранилищем будущих результатов цикла 
Int=[0,0,0,0]
Abs=[0,0,0,0]
White=[0,0,0,0]
Black=[0,0,0,0]
Conc_r=[0,0,0,0]
AxA=[0,0,0,0]
AxA_obr=[0,0,0,0]
CxA=[0,0,0,0]
B=[0,0,0,0]
dC=[0,0,0,0]
dC_FK=[0,0,0,0]
dC_BS=[0,0,0,0]
Conc=[0,0,0,0]

#отображение с двумя цифрами после запятой
np.set_printoptions(suppress=True,precision=2)

#вводим матрицу концентраций
C=np.genfromtxt(put/'конц.txt')

for j in range (4):

  #объединяем 5 параллельных измерений в одно усредненное. Получаем файл с интенсивностями
  Int[j]=np.zeros((3,72))
  if j!=3:
    for i in range(j*5+1,j*5+6):
      a=np.genfromtxt(put2/ f'({i}).csv',delimiter=',')
      Int[j]+=a[1:,1:73]
    Int[j]/=5
  else:
    Int[j]=np.genfromtxt(put2/ f'({j*5+1}).csv',delimiter=',')[1:,1:]

  #выводим значения туши и воды из данных
  White[j]=np.mean(Int[j][:,-4:-1],axis=1,keepdims=True)
  Black[j]=np.mean(Int[j][:,-8:-4],axis=1,keepdims=True)
  Int[j]=Int[j][:,:-8]

  #отсеиваем мусорные значения, где светлоты выпали из фотографии
  if j==0:
    Matrix=np.vstack([C,Int[j]])
    Matrix=np.delete(Matrix,np.where(Matrix[2:]<2300)[1],1)
    Conc[j]=Matrix[:2]
    Int[j]=Matrix[2:]
  else:
    Matrix=np.vstack([C,Int[j]])
    Conc[j]=Matrix[:2]
    Int[j]=Matrix[2:]

  #вводим матрицу оптических плотностей
  Abs[j]=np.zeros_like(Int[j])
  for i in range(3):
    Abs[j][i,:]=np.log10((White[j][i,0]- Black[j][i,0])/(Int[j][i,:]- Black[j][i,0]))

  #отсеиваем мусорные значения, где оптические плотности не входят в диапазон 0,01-0,9
  Matrix=np.vstack([Conc[j],Abs[j]])
  #Matrix=np.delete(Matrix,np.where(np.logical_or(Matrix[2:]<0.01,Matrix[2:]>0.9))[1],1)
  Conc[j]=Matrix[:2]
  Abs[j]=Matrix[2:]

  #делаем матрицу пересчетов по формулкам
  AxA=Abs[j]@Abs[j].T
  AxA_obr=np.linalg.inv(AxA)
  CxA=Conc[j]@Abs[j].T
  B[j]=CxA@AxA_obr
  
  #считаем концентрации по расчетам
  Conc_r[j]=B[j]@Abs[j]
  
  #относительные погрешности расчетов
  dC[j]=np.abs((Conc[j]-Conc_r[j])/Conc[j]*100)
  
  

  haha = pd.DataFrame(Conc_r[j].T)
  
  haha.columns=["ФК","БС"]
  haha["ФК - реал"] = Conc[j][0].T
  haha["БС - реал"] = Conc[j][1].T
  
  print(haha.groupby(["ФК - реал"])[["ФК"]].agg(["count",np.mean,np.std,np.min,np.max]))
  print(haha.groupby(["БС - реал"])[["БС"]].agg(["count",np.mean,np.std,np.min,np.max]))

  #сохраняем файлы со статистикой  в csv формате
  #haha.groupby(["ФК - реал"])[["ФК"]].agg(["count",np.mean,np.std,np.min,np.max]).to_csv('ФК'+str(320//(2**j))+'.csv')
  #haha.groupby(["БС - реал"])[["БС"]].agg(["count",np.mean,np.std,np.min,np.max]).to_csv('БС'+str(320//(2**j))+'.csv')

             ФК                                        
          count      mean       std      amin      amax
ФК - реал                                              
0.000006      8  0.000006  0.000014 -0.000019  0.000019
0.000013      8  0.000012  0.000016 -0.000012  0.000033
0.000026      8  0.000034  0.000018  0.000010  0.000055
0.000052      8  0.000062  0.000016  0.000038  0.000080
0.000103      8  0.000126  0.000017  0.000097  0.000150
0.000206      8  0.000240  0.000021  0.000202  0.000268
0.000413      6  0.000473  0.000026  0.000434  0.000502
0.000825      6  0.000763  0.000030  0.000711  0.000792
             БС                                            
          count          mean       std      amin      amax
БС - реал                                                  
0.000001      8  1.643052e-07  0.000005 -0.000008  0.000007
0.000002      8  4.714089e-06  0.000006 -0.000005  0.000012
0.000005      8  4.976501e-06  0.000004 -0.000002  0.000010
0.000010      6  1.35772

In [72]:
from sklearn import metrics

In [78]:
for  i in range(4):
    print('MSE:', metrics.mean_squared_error(Conc[i], Conc_r[i]))
    print('R²: ', metrics.r2_score(Conc[i], Conc_r[i]))

MSE: 6.964960528739252e-10
R²:  -1.7707666040427508
MSE: 9.841758493160757e-11
R²:  0.6705421699296434
MSE: 3.2410907572383775e-11
R²:  0.10665033343327546
MSE: 3.940703334965819e-09
R²:  -714.5020569372219


In [150]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Abs[1].T, Conc[1].T,
        shuffle=True, random_state=1240, train_size=0.8)
rgr1 = MultiOutputRegressor(RandomForestRegressor(n_estimators=5000, max_depth=12))
rgr1.fit(x_train, y_train)
y1_predict = rgr1.predict(x_test)
print('MSE:', metrics.mean_squared_error(y_test, y1_predict))
print('R²: ', metrics.r2_score(y_test, y1_predict))
haha = pd.DataFrame(y1_predict)
haha.columns = ['ФК','БС']
haha["ФК - реал"] = y_test[:,0]
haha["БС - реал"] = y_test[:,1]
print(haha.groupby(["ФК - реал"])[["ФК"]].agg(["count",np.mean,np.std,np.min,np.max]))
print(haha.groupby(["БС - реал"])[["БС"]].agg(["count",np.mean,np.std,np.min,np.max]))

MSE: 3.750136722233603e-09
R²:  0.94448888129606
             ФК                                        
          count      mean       std      amin      amax
ФК - реал                                              
0.000006      1  0.000014       NaN  0.000014  0.000014
0.000026      2  0.000041  0.000033  0.000018  0.000065
0.000052      2  0.000070  0.000043  0.000040  0.000101
0.000103      1  0.000056       NaN  0.000056  0.000056
0.000206      1  0.000072       NaN  0.000072  0.000072
0.000413      4  0.000384  0.000121  0.000218  0.000504
0.000825      2  0.000724  0.000067  0.000676  0.000771
             БС                                            
          count      mean           std      amin      amax
БС - реал                                                  
0.000001      3  0.000003  1.074195e-06  0.000002  0.000004
0.000002      1  0.000006           NaN  0.000006  0.000006
0.000005      1  0.000003           NaN  0.000003  0.000003
0.000010      1  0.000007      

In [151]:
haha["ФК %"] = np.abs(y_test[:,0]-y1_predict[:,0])/y1_predict[:,0]*100
haha["БС %"] = np.abs(y_test[:,1]-y1_predict[:,1])/y1_predict[:,1]*100

print(haha["ФК %"].mean(), haha["БС %"].mean(),) 
haha.head(80)

50.31890518032587 37.38430955811733


,ФК,БС,ФК - реал,БС - реал,ФК %,БС %
0,0.000429,0.000004,0.000413,0.000001,3.907078,72.023029
1,0.000065,0.000157,0.000026,0.000157,60.129181,0.120144
2,0.000056,0.000003,0.000103,0.000005,83.414724,70.854500
3,0.000218,0.000011,0.000413,0.000020,89.043734,78.652886
4,0.000101,0.000078,0.000052,0.000078,48.955502,0.735368
5,0.000385,0.000088,0.000413,0.000078,7.254780,10.506533
6,0.000771,0.000039,0.000825,0.000020,7.043460,50.178113
7,0.000072,0.000082,0.000206,0.000078,186.774327,4.443383
8,0.000504,0.000006,0.000413,0.000002,18.188698,57.883070
9,0.000040,0.000003,0.000052,0.000001,29.546264,56.364371
